In [1]:
import akshare as ak
import pandas as pd

In [ ]:
y2601_df = ak.futures_main_sina(symbol='Y2601')

In [ ]:
print(y2601_df.tail(20))

In [ ]:
# 1. 获取主力连续合约数据（以螺纹钢为例，symbol通过ak.futures_display_main_sina()确认）
import pandas as pd


symbol = "RB0"  # 螺纹钢主力连续
main_df = ak.futures_main_sina(symbol=symbol, start_date="20180101", end_date="20231231")
print(main_df.head())
# main_df["日期"] = pd.to_datetime(main_df["日期"])
# main_df = main_df.sort_values("日期").set_index("日期")

# # 2. 定位换月点（价格跳空超过阈值的日期，阈值根据品种特性调整）
# main_close = main_df["收盘价"]
# # 计算单日涨跌幅绝对值（跳空幅度）
# price_jump = main_close.pct_change().abs()
# # 筛选跳空超过5%的日期作为候选换月点（螺纹钢通常波动较小，5%可有效识别换月）
# candidate_dates = price_jump[price_jump > 0.05].index.tolist()
# print(f"候选换月点：{candidate_dates}")

In [ ]:


# 获取主力连续合约数据
df = ak.futures_main_sina(symbol="RB0", start_date="20200101", end_date="20240101")
df["日期"] = pd.to_datetime(df["日期"])

# 计算相邻交易日的收盘价差
df["价差"] = df["收盘价"].diff()

# 筛选出显著跳空（例如价差绝对值超过100点）
jump_gaps = df[abs(df["价差"]) > 100]
print("换月跳空情况：")
print(jump_gaps[["日期", "收盘价", "价差"]].tail(20))

In [ ]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime

def futures_main_weekly_sina(
    symbol: str = "V0",
    start_date: str = "19900101",
    end_date: str = "22220101",
) -> pd.DataFrame:
    """
    新浪财经-期货-主力连续周线数据
    基于日线数据聚合计算，或直接调用周线接口
    """
    # 1. 获取日线数据后聚合为周线（通用方法）
    daily_df = ak.futures_main_sina(symbol=symbol, start_date=start_date, end_date=end_date)
    daily_df["日期"] = pd.to_datetime(daily_df["日期"])
    daily_df.set_index("日期", inplace=True)
    
    # 聚合逻辑：每周一至周五为一周，取首开、最高、最低、尾收、成交量和持仓量总和
    weekly_df = daily_df.resample("W-FRI").agg({
        "开盘价": "first",
        "最高价": "max",
        "最低价": "min",
        "收盘价": "last",
        "成交量": "sum",
        "持仓量": "sum",
        "动态结算价": "last"
    }).dropna()
    
    # 2. 日期筛选
    weekly_df = weekly_df[(weekly_df.index >= pd.to_datetime(start_date)) & 
                         (weekly_df.index <= pd.to_datetime(end_date))]
    weekly_df.reset_index(inplace=True)
    weekly_df.rename(columns={"index": "日期"}, inplace=True)
    return weekly_df

# 示例调用
weekly_data = futures_main_weekly_sina(symbol="V0", start_date="20240101", end_date="20251231")
print(weekly_data)

In [ ]:
# 1. 提取品种代码（如"RB0"的核心代码是"RB"）
base_symbol = symbol[:-1]  # 截取前两位，"RB0"→"RB"

# 2. 确定时间范围（主力连续数据的起始年份到结束年份）
start_year = main_df.index.min().year
end_year = main_df.index.max().year

# 3. 生成该时间范围内的所有可能合约（每年1-12月）
contracts = []
for year in range(start_year, end_year + 1):
    year_code = str(year)[-2:]  # 年份后两位（2023→23）
    for month in range(1, 13):
        month_code = f"{month:02d}"  # 月份补0（1→01）
        contract = f"{base_symbol}{year_code}{month_code}"
        contracts.append(contract)
print(f"生成可能的次合约列表（共{len(contracts)}个）：{contracts[:5]}...")

In [ ]:
print(y2601_df.head())
print(y2601_df.tail())

### 验证主力合约是否存在跳空问题

In [ ]:
jm0_df = ak.futures_main_sina(symbol='jm0', start_date= '20250101', end_date='20251231')
print(jm0_df.head(30))
print(jm0_df.tail())

In [2]:
daily_df = ak.futures_zh_daily_sina(symbol='CJ2601')
print(daily_df.head())
print(daily_df.tail())

         date     open     high      low    close  volume  hold   settle
0  2025-01-16  10375.0  10500.0  10290.0  10465.0     563   375  10425.0
1  2025-01-17  10505.0  10550.0  10430.0  10475.0     311   518  10500.0
2  2025-01-20  10405.0  10515.0  10400.0  10495.0     164   561  10465.0
3  2025-01-21  10530.0  10555.0  10455.0  10490.0     138   595  10500.0
4  2025-01-22  10455.0  10540.0  10435.0  10505.0     147   666  10490.0
           date     open     high      low    close  volume    hold   settle
186  2025-10-28  10420.0  10530.0  10310.0  10445.0  265154  185560  10415.0
187  2025-10-29  10460.0  10565.0  10375.0  10495.0  199932  184472  10485.0
188  2025-10-30  10500.0  10525.0  10175.0  10225.0  324621  188572  10295.0
189  2025-10-31  10285.0  10375.0  10120.0  10145.0  245521  180474  10260.0
190  2025-11-03  10235.0  10340.0  10185.0  10280.0  162931  178859  10265.0


: 

In [ ]:
print(ak.__version__)

In [ ]:
def ensure_directory_exists(file_path):
    """
    确保文件路径中的目录存在，如果不存在则创建（支持多级嵌套目录）
    """
    from pathlib import Path
    
    # 使用 pathlib 更可靠地处理路径
    path = Path(file_path)
    
    # 创建所有父目录（如果不存在）
    path.parent.mkdir(parents=True, exist_ok=True)
    
    print(f"✅ 已确保目录存在: {path.parent}")
    return True

In [ ]:
# 测试各种嵌套目录情况
test_cases = [
    "reports/report_20251101_235144/1.txt",                    # 1层嵌套
]

for test_path in test_cases:
    ensure_directory_exists(test_path)